In [1]:
import os
from datetime import datetime

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/Othercomputers/Il mio MacBook Pro/MagicKnob
%ls

/content/drive/Othercomputers/Il mio MacBook Pro/MagicKnob
build/  CMakeLists.txt  data/  python/  README.md  src/


In [3]:
drive = True
if drive:
    path = "/content/drive/Othercomputers/Il mio MacBook Pro/MagicKnob/"

    # check python file folder
    assert os.path.exists(path + "python"), f"Upload python files in {path}python"
    %cd ./python

    # check data folder
    assert os.path.exists(path + "data"), f"Upload data files in {path}data"

/content/drive/Othercomputers/Il mio MacBook Pro/MagicKnob/python


In [4]:
import myk_data_conv
import myk_models_conv
import myk_loss
import myk_train_conv
import torch
from torch.utils.data import DataLoader
import myk_evaluate_conv
import os

In [5]:
from torch.utils.tensorboard import SummaryWriter

In [6]:
# used for the writing of example outputs
run_name="audio_conv1d_phaser"
# dataset : need an input and output folder in this folder
audio_folder = f"../data/{run_name}"
#audio_folder = "../../data/audio_ht1"
assert os.path.exists(audio_folder), f"Audio folder  not found. Looked for {audio_folder}"
# used to render example output during training
test_file = "../data/guitar.wav"
assert os.path.exists(test_file), "Test file not found. Looked for " + test_file

In [7]:
# initialize net specs
lstm_hidden_size = 32
learning_rate = 5e-3
batch_size = 50
max_epochs = 10000

# create the logger for tensorboard
writer = SummaryWriter()

In [1]:
print("Loading dataset from folder ", audio_folder)
dataset = myk_data_conv.generate_dataset(audio_folder + "/input/", audio_folder + "/output/", frag_len_seconds=0.5)

print("Splitting dataset")
train_ds, val_ds, test_ds = myk_data_conv.get_train_valid_test_datasets(dataset)

NameError: name 'audio_folder' is not defined

In [9]:
dataset[0]

(tensor([[-3.0518e-05],
         [-3.0518e-05],
         [ 0.0000e+00],
         ...,
         [ 0.0000e+00],
         [ 0.0000e+00],
         [ 0.0000e+00]]),
 tensor([[ 0.0000e+00],
         [ 0.0000e+00],
         [ 0.0000e+00],
         ...,
         [-3.8624e-05],
         [-2.5749e-05],
         [-1.6809e-05]]))

In [10]:
# test GPU, must be done after splitting
device = myk_train_conv.get_device()

cuda device available


In [11]:
# create data loaders
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, generator=torch.Generator(device=device))
val_dl = DataLoader(val_ds, batch_size=batch_size, shuffle=True, generator=torch.Generator(device=device))
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=True, generator=torch.Generator(device=device))

In [12]:
model = myk_models_conv.SimpleConv1D().to(device)

In [13]:
# crate optimizer and loss function
optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimiser, 'min', factor=0.5, patience=5, verbose=True) non lo usava

loss_functions = myk_loss.LossWrapper()

# https://github.com/Alec-Wright/Automated-GuitarAmpModelling/blob/main/dist_model_recnet.py
# https://github.com/Alec-Wright/CoreAudioML/blob/bad9469f94a2fa63a50d70ff75f5eff2208ba03f/training.py

In [14]:
#%load_ext tensorboard
#%tensorboard --logdir logs

In [15]:
# training loop
lowest_val_loss = 0
best_loss = False

max_patience = 500
curr_patience = max_patience

# datetime object containing current date and time
best_epoch = 0
now = datetime.now()
dt_string = now.strftime(f"models/{model.model_type}_%d-%m-%Y_%H-%M-%S")

os.mkdir(os.path.join(".", dt_string))

for epoch in range(max_epochs):
    #ep_loss = myk_train.train_epoch_interval(model, train_dl, loss_functions, optimiser, device=device)
    ep_loss = myk_train_conv.train_epoch_conv(model, train_dl, loss_functions, optimiser, device=device)

    #ep_loss = myk_train.train_epoch(model, train_dl, loss_functions, optimiser, device=device)
    val_loss = myk_train_conv.compute_batch_loss(model, val_dl, loss_functions, device=device)
    writer.add_scalar("Loss/val", val_loss, epoch)
    writer.add_scalar("Loss/train", ep_loss, epoch)

    # check if we have beaten our best loss to date
    if lowest_val_loss == 0:# first run
        lowest_val_loss = val_loss
    elif val_loss < lowest_val_loss:# new record
        lowest_val_loss = val_loss
        best_loss = True
    else: # no improvement
        best_loss = False
        curr_patience -= 1

    if best_loss: # save best model so far
        best_epoch = epoch
        print(f"    Record loss - saving at epoch {epoch}")
        # save for RTNeural
        model.save_for_rtneural(f"{dt_string}/model.json")
        # save for pythorch
        torch.save(model.state_dict(), f"{dt_string}/model.ph")
        print(f"    epoch {epoch}, train_loss {ep_loss}, val_loss {val_loss} ")
        curr_patience = max_patience
    if epoch % 50 == 0: # save an example processed audio file
        myk_evaluate_conv.run_file_through_model(model, test_file, audio_folder + "/" + run_name + str(epoch)+".wav")
        print(f"epoch {epoch}, train_loss {ep_loss}, val_loss {val_loss} ")
    if curr_patience == 0:
        print("max patience reached, stopping training")
        # load best parameters in the model
        model.load_state_dict(torch.load(f"{dt_string}/model.ph"))
        model.eval() # set inference state in the possible layers that need it
        myk_evaluate_conv.run_file_through_model(model, test_file, audio_folder + "/" + run_name + str(best_epoch)+"_BEST.wav")
        break

epoch 0, train_loss 0.7835885882377625, val_loss 0.6886797547340393 
    Record loss - saving at epoch 1
    epoch 1, train_loss 0.6747114658355713, val_loss 0.6677094101905823 
    Record loss - saving at epoch 2
    epoch 2, train_loss 0.6624307036399841, val_loss 0.6546810865402222 
    Record loss - saving at epoch 3
    epoch 3, train_loss 0.6432195901870728, val_loss 0.6436448097229004 
    Record loss - saving at epoch 4
    epoch 4, train_loss 0.6411299705505371, val_loss 0.6394931077957153 
    Record loss - saving at epoch 5
    epoch 5, train_loss 0.6374751329421997, val_loss 0.633129358291626 
    Record loss - saving at epoch 6
    epoch 6, train_loss 0.6337488889694214, val_loss 0.6250375509262085 
    Record loss - saving at epoch 8
    epoch 8, train_loss 0.6134675145149231, val_loss 0.6156945824623108 
    Record loss - saving at epoch 14
    epoch 14, train_loss 0.6132434606552124, val_loss 0.6136554479598999 
    Record loss - saving at epoch 15
    epoch 15, train_l

In [16]:
to_append = [f'\n{run_name}/*.wav', f'\n!{run_name}/*_BEST.wav']

with open(f'{audio_folder}/../.gitignore', "r+") as gitignore:
    read = gitignore.read()
    if to_append[0][2:len(to_append[0])] not in read:
        gitignore.writelines(to_append)